# **Classificadores: Regressão Logística, Árvore de Decisão e Naive Bayes**

## Disciplina: Inteligência Artificial e Aplicações
## FATEC Ferraz de Vasconcelos
### Professora: Ana Rosa C. Tonão
#### Data: 23/06/2025

-----------------------

# **Comparação de Modelos de Classificação para Previsão de Churn**

**Objetivo:** Este notebook tem como objetivo treinar, avaliar e comparar três diferentes algoritmos de classificação para o problema de previsão de churn de clientes da MegaTelCo.

Os modelos a serem avaliados são:
1.  Regressão Logística (nosso modelo base)
2.  Árvore de Decisão
3.  Gaussian Naïve Bayes

A comparação final será realizada com base em métricas de performance como Acurácia, Precisão, Recall, F1-Score e, principalmente, a AUC (Área Sob a Curva ROC).

--------------------


### **Importando as Bibliotecas Necessárias**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Pré-processamento
from sklearn.preprocessing import StandardScaler

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

# Métricas de Avaliação
from sklearn.metrics import accuracy_score, classification_report, roc_curve, roc_auc_score

# Configurações visuais
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

### **Carregamento e Preparação dos Dados**

In [2]:
import os
os.chdir('C:\\Code\\ML\\Dados')
os.getcwd()

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'C:\\Code\\ML\\Dados'

In [ ]:
# Carregando os dados
retention_train = pd.read_csv('train_deduplicado.csv')
retention_test = pd.read_csv('test_deduplicado.csv')

In [ ]:
# Crie um dicionário para o mapeamento desejado
label_map = {
    'STAY': 0,
    'LEAVE': 1
}

In [ ]:
# Aplique o mapeamento à sua coluna usando o método .map() do pandas
retention_train['leave'] = retention_train['leave'].map(label_map)

In [ ]:
retention_train

In [ ]:
# Aplique o mapeamento à sua coluna usando o método .map() do pandas
retention_test['leave'] = retention_test['leave'].map(label_map)

In [ ]:
retention_test

### **Preparação dos Dados**

In [ ]:
# --- Pré-processamento ---

# 1. Tratamento de valores faltantes (usando a média do TREINO para ambos)
media_preco_treino = retention_train['handset_price'].mean()
retention_train['handset_price'] = retention_train['handset_price'].fillna(media_preco_treino)
retention_test['handset_price'] = retention_test['handset_price'].fillna(media_preco_treino)

# 2. Definindo as features (X) e a variável alvo (y)
features = ['income', 'house', 'handset_price', 'overage', 'leftover', 'over_15mins_calls_per_month', 'average_call_duration']
target = 'leave'

X_train = retention_train[features]
y_train = retention_train[target]

X_test = retention_test[features]
y_test = retention_test[target]

# 3. Escalonamento das Features
# É uma boa prática para Regressão Logística e Naive Bayes
scaler = StandardScaler()

# Ajustamos o scaler com os dados de TREINO e transformamos ambos os conjuntos
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Dados prontos para o treinamento!")
print("Formato de X_train_scaled:", X_train_scaled.shape)
print("Formato de X_test_scaled:", X_test_scaled.shape)

## **2. Treinando os Classificadores**

Agora, vamos instanciar e treinar cada um dos quatro modelos com nossos dados de treino escalonados.

In [ ]:
# Dicionário para armazenar os modelos treinados
models = {}

# --- 1. Regressão Logística (Baseline) ---
log_reg = LogisticRegression(solver='liblinear', random_state=42)
log_reg.fit(X_train_scaled, y_train)
models['Regressão Logística'] = log_reg
print("Modelo 'Regressão Logística' treinado.")

# --- 2. Árvore de Decisão ---
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train_scaled, y_train)
models['Árvore de Decisão'] = tree_clf
print("Modelo 'Árvore de Decisão' treinado.")

# --- 3. Gaussian Naïve Bayes ---
# Ideal para features contínuas que assumimos ter uma distribuição normal
gnb = GaussianNB()
gnb.fit(X_train_scaled, y_train)
models['Naïve Bayes'] = gnb
print("Modelo 'Naïve Bayes' treinado.")


## **3. Comparação de Performance**

Com os modelos treinados, o próximo passo é avaliá-los nos dados de teste. Faremos isso de duas formas:
1.  **Tabela de Métricas:** Um resumo com as principais métricas de classificação.
2.  **Curva ROC:** Uma comparação visual da capacidade de cada modelo em distinguir as classes.

In [ ]:
# Dicionário para armazenar os resultados de cada modelo
performance_data = {}

# Loop para calcular as métricas de cada modelo
for name, model in models.items():
    # Fazer previsões nos dados de teste
    y_pred = model.predict(X_test_scaled)
    
    # Gerar o classification report como um dicionário
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Calcular a AUC
    # Para AUC, precisamos da probabilidade da classe positiva (1)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)

    print(report)
    
    # Salvar as métricas de interesse (focando na classe '1', que é o churn)
    performance_data[name] = {
        'Acurácia': report['accuracy'],
        'Precisão (classe 1)': report['1']['precision'],
        'Recall (classe 1)': report['1']['recall'],
        'F1-Score (classe 1)': report['1']['f1-score'],
        'AUC': auc
    }

# Converter o dicionário em um DataFrame do pandas para melhor visualização
performance_df = pd.DataFrame.from_dict(performance_data, orient='index')

# Exibindo a tabela de performance ordenada pela AUC (melhor métrica geral)
print("Tabela Comparativa de Performance dos Modelos:")
display(performance_df.sort_values(by='AUC', ascending=False))

In [ ]:
print(report)

### **Comparação Visual com a Curva ROC**

In [ ]:
plt.figure(figsize=(12, 8))

# Loop para plotar a Curva ROC de cada modelo
for name, model in models.items():
    # Prever as probabilidades
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # Calcular fpr, tpr e AUC
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    # Plotar a curva
    plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.4f})')

# Plotar a linha de referência (classificador aleatório)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--', label='Classificador Aleatório')

# Configurações do gráfico
plt.title('Comparação da Curva ROC entre os Modelos', fontsize=16)
plt.xlabel('Taxa de Falsos Positivos (FPR)', fontsize=12)
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)', fontsize=12)
plt.legend(loc='lower right')
plt.show()

## **4. Conclusão**

Ao analisar a tabela de métricas e o gráfico da Curva ROC, podemos tirar algumas conclusões:

* **Melhor Modelo Geral (AUC):** O modelo com a maior AUC foi o **[Nome do Modelo com Maior AUC]**. A AUC é uma ótima métrica geral porque avalia a capacidade do modelo de discriminar corretamente entre clientes que sairão e que ficarão, em todos os limiares de probabilidade.

* **Melhor em Evitar Falsos Positivos (Precisão):** Se o objetivo da MegaTelCo fosse gastar recursos de retenção apenas com clientes que **com certeza** vão sair (evitando "incomodar" clientes fiéis), o modelo com a maior **precisão** para a classe 1 seria o mais indicado. Esse modelo foi o **[Nome do Modelo com Maior Precisão]**.

* **Melhor em "Pegar" Todos os Churns (Recall):** Se a prioridade fosse identificar o maior número possível de clientes que estão prestes a sair, mesmo que isso signifique incluir alguns alarmes falsos, o modelo com o maior **recall** seria a escolha. Esse modelo foi o **[Nome do Modelo com Maior Recall]**.


**Recomendação Final:** Com base nos resultados, o **[Nome do Modelo Vencedor]** parece ser a escolha mais equilibrada e performática para este problema.